# Detection with CNNClassifier


In [1]:
from dotenv import load_dotenv

load_dotenv()

import sys
import os

sys.path.append(os.getenv('SRC_DIR'))

from utils.system import display_system_info

display_system_info(markdown=True)


**Last Updated**: 2025-04-27 19:20:11

**Python Version**: 3.11.12  
**OS**: Windows 10.0.26100  
**Architecture**: 64bit  
**Hostname**: ShenLaptop  
**Processor**: Intel64 Family 6 Model 186 Stepping 3, GenuineIntel  
**RAM Size**: 15.65 GB  
  
        

In [2]:
from torchvision import transforms
import torch
import cv2
from PIL import Image
import numpy as np

from classify import TorchClassifier

In [15]:
IDX_TO_CLASS = {
    0: 'swimming',
    1: 'treadwater',
    2: 'drowning'
}

def xywhn_to_xyxy(image, x_n, y_n, w_n, h_n):
    # cv2 images in HWC
    height, width, _ = image.shape

    x1 = int((x_n - w_n / 2) * width)
    y1 = int((y_n - h_n / 2) * height)
    x2 = int((x_n + w_n / 2) * width)
    y2 = int((y_n + h_n / 2) * height)

    return x1, y1, x2, y2

def cop_save_roi(images_folder, labels_folder, output_folder, idx_to_class):
    os.makedirs(output_folder, exist_ok=True)
    for class_id, class_name in idx_to_class.items():
        os.makedirs(os.path.join(output_folder, class_name), exist_ok=True)
        
    class_counters = {class_id: 0 for class_id in idx_to_class.keys()}

    for image_file in os.listdir(images_folder):
        if image_file.endswith('.jpg'):
            base_name = os.path.splitext(image_file)[0]
            
            label_file = os.path.join(labels_folder, f"{base_name}.txt")
            image_path = os.path.join(images_folder, image_file)
            
            image = cv2.imread(image_path)
            
            # Read the label file
            with open(label_file, 'r') as file:
                lines = file.readlines()
            
            # Loop through each line in the label file
            for line in lines:
                values = line.strip().split()
                class_id = int(values[0])
                x_n, y_n, w_n, h_n = map(float, values[1:])

                x1, y1, x2, y2 = xywhn_to_xyxy(image, x_n, y_n, w_n, h_n)

                roi = image[y1:y2, x1:x2]

                cv2.imwrite(os.path.join(output_folder, idx_to_class[class_id], f"{str(class_counters[class_id]).zfill(6)}.jpg"), roi)

                class_counters[class_id] += 1

In [ ]:
for split in ['train', 'val']:
    images_folder = os.path.join(os.getenv('RAW_DATA_DIR'), 'images', split)
    labels_folder = os.path.join(os.getenv('RAW_DATA_DIR'), 'labels', split)
    output_folder = os.path.join(os.getenv('ROI_DATA_DIR'), split)

    cop_save_roi(images_folder, labels_folder, output_folder, IDX_TO_CLASS)

In [4]:
model_config = {
    'num_classes': 3,
    'num_blocks': 4,
    'first_out_channel': 32,
    'out_channel_multiplier': 2,
    'kernel_size': 3,
    'stride': 1,
    'padding': 1,
    'input_shape': (3, 128, 128)
}

model = TorchClassifier(model='CNNClassifier', config=model_config, device='auto', verbose=True)

New model created: CNNClassifier(
  (features): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (2): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, trac

In [3]:
class RandomAffineWithInpainting:
    def __init__(self, degrees=10, translate=(0.1, 0.1), radius=3):
        self.affine = transforms.RandomAffine(degrees=degrees, translate=translate)
        self.inpaint_radius = radius

    def __call__(self, img):
        # Step 1: Apply affine transform (returns PIL image)
        img = self.affine(img)

        # Step 2: Convert to NumPy array
        img_np = np.array(img)

        return Image.fromarray(img_np)

        # Step 3: Create mask where pixels are black
        gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
        mask = cv2.inRange(gray, 0, 1)

        # Step 4: Inpaint using OpenCV
        inpainted_np = cv2.inpaint(img_np, mask, self.inpaint_radius, cv2.INPAINT_TELEA)

        # Step 5: Convert back to PIL
        img_inpainted = Image.fromarray(inpainted_np)

        return img_inpainted
    
class RGBToHSV:
    def __call__(self, img):
        # Ensure the image is in PIL format before converting
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)  # Convert tensor to PIL image
        
        # Convert the image to HSV using PIL
        img_hsv = img.convert("HSV")
        
        return img_hsv
    
class VBlurring:
    def __call__(self, img):
        # Ensure the image is in PIL format before converting
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)

        # apply blurring to the V channel of the HSV image
        h, s, v = cv2.split(np.array(img))

        v_blurred = cv2.blur(v, (7, 7))  # Apply 3x3 average blur to the V channel

        img_hsv_blurred = cv2.merge([h, s, v_blurred])

        return Image.fromarray(img_hsv_blurred)

aug_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1.0),
    RandomAffineWithInpainting(degrees=0, translate=(0.2, 0.2))
])

enhance_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    RGBToHSV(),
    VBlurring(),
    # May addd certain non-deterministic augmentations here to improve generalisability
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    RGBToHSV(),
    transforms.ToTensor()
])

In [ ]:
results = model.train(
    data_path=os.getenv('ROI_DATA_DIR'),
    imbalance=True,
    fraction=1,
    val_test_ratio=0.5,
    input_size=128,
    optimizer='Adam',
    lr=1e-4,
    aug_transform=aug_transform,
    enhance_transform=enhance_transform,
    val_transform=val_transform,
    batch_size=32,
    epochs=15
) 

      Epoch       Loss   Accuracy    Macro(P          R        F1) Weighted(P          R        F1)


       1/15     0.3291      87.08     0.8707     0.8713      0.871     0.8707     0.8713      0.871: 100%|██████████| 552/552 [00:39<00:00, 13.86it/s]


                  Loss


                0.1286: 100%|██████████| 37/37 [00:01<00:00, 24.29it/s]


              precision    recall  f1-score   support

    drowning     0.9405    0.8681    0.9029       182
    swimming     0.9756    0.9841    0.9798       690
  treadwater     0.9113    0.9368    0.9239       285

    accuracy                         0.9542      1157
   macro avg     0.9424    0.9297    0.9355      1157
weighted avg     0.9542    0.9542    0.9539      1157

      Epoch       Loss   Accuracy    Macro(P          R        F1) Weighted(P          R        F1)


...

              precision    recall  f1-score   support

    drowning     0.9770    0.9341    0.9551       182
    swimming     0.9913    0.9855    0.9884       690
  treadwater     0.9428    0.9825    0.9622       285

    accuracy                         0.9767      1157
   macro avg     0.9703    0.9673    0.9685      1157
weighted avg     0.9771    0.9767    0.9767      1157

      Epoch       Loss   Accuracy    Macro(P          R        F1) Weighted(P          R        F1)


      15/15    0.07553      97.29     0.9734     0.9734     0.9734     0.9734     0.9734     0.9734: 100%|██████████| 552/552 [00:41<00:00, 13.32it/s]


                  Loss


                0.0798: 100%|██████████| 37/37 [00:01<00:00, 24.53it/s]

              precision    recall  f1-score   support

    drowning     0.9774    0.9505    0.9638       182
    swimming     0.9913    0.9870    0.9891       690
  treadwater     0.9556    0.9825    0.9689       285

    accuracy                         0.9801      1157
   macro avg     0.9748    0.9733    0.9739      1157
weighted avg     0.9803    0.9801    0.9801      1157



In [6]:
results

([{'loss': 0.3291142154754936,
   'accuracy': 87.08342779415099,
   'macro_f1': 0.870964555944839,
   'weighted_f1': 0.8709645559448391,
   'macro_precision': 0.8707308151546386,
   'weighted_precision': 0.8707308151546386,
   'macro_recall': 0.8713281161392764,
   'weighted_recall': 0.8713281161392764},
  {'loss': 0.18564937731075654,
   'accuracy': 93.32350940829744,
   'macro_f1': 0.9337688310940582,
   'weighted_f1': 0.9337688310940583,
   'macro_precision': 0.9337750740331048,
   'weighted_precision': 0.9337750740331048,
   'macro_recall': 0.9337643189293411,
   'weighted_recall': 0.9337643189293411},
  {'loss': 0.16269371990178563,
   'accuracy': 94.27000680117887,
   'macro_f1': 0.9432435492403695,
   'weighted_f1': 0.9432435492403695,
   'macro_precision': 0.9432749495171722,
   'weighted_precision': 0.9432749495171721,
   'macro_recall': 0.9432346603153,
   'weighted_recall': 0.9432346603153},
  {'loss': 0.1514506953569345,
   'accuracy': 94.88778054862843,
   'macro_f1': 0.94

In [8]:
model.save(
    os.path.join(os.getenv('CNN_MODEL_DIR'), 'test.pt')
)

Model saved to C:/Users/PC/Downloads/Jie Shen/Drowning-Detection/models/classification/CNN\test.pt


In [7]:
# Add model prediction here
loaded_model = TorchClassifier(model='CNNClassifier', model_path=os.path.join(os.getenv('CNN_MODEL_DIR'), 'test.pt'), device='auto', verbose=False)

img = Image.open(rf'C:\Users\hp\Downloads\Drowning-Detection\data\roi\val\drowning\{str(63).zfill(6)}.jpg')

class RGBToHSV:
    def __call__(self, img):
        # Ensure the image is in PIL format before converting
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)  # Convert tensor to PIL image
        
        # Convert the image to HSV using PIL
        img_hsv = img.convert("HSV")
        
        return img_hsv

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    RGBToHSV(),
    transforms.ToTensor()
])

prediction = loaded_model(
    img=img,
    transform=val_transform,
    prob=True
).cpu().numpy()

IDX_TO_CLASS2 = {
    0: 'drowning',
    1: 'swimming',
    2: 'treadwater'
}

print(prediction)
print(IDX_TO_CLASS2[np.argmax(prediction)])

loaded_model_results = loaded_model.validate(
    data_path=os.getenv('ROI_DATA_DIR'),
    transform=val_transform,
    val_test_ratio=0.5
)

[0.6086047  0.12980805 0.26158726]
drowning


Validation: 100%|██████████| 37/37 [00:16<00:00,  2.27it/s]

              precision    recall  f1-score   support

    drowning     0.9730    0.9890    0.9809       182
    swimming     0.9913    0.9942    0.9928       690
  treadwater     0.9786    0.9614    0.9699       285

    accuracy                         0.9853      1157
   macro avg     0.9810    0.9815    0.9812      1157
weighted avg     0.9853    0.9853    0.9853      1157

